In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    import os
    os.chdir('/content')
    if not os.path.isdir('/content/EVA-2-Group/'):
        !git clone https://github.com/sambitdash/EVA-2-Group.git
    os.chdir('/content/EVA-2-Group/Session-19')
    !pwd
    
    !git config user.email "sambitdash@gmail.com"
    !git config user.name "Sambit Kumar Dash"
    !git config user.password "your password"
    !git status

In [ ]:
import cv2
import os
import numpy as np
from os import listdir
from os.path import join

car_types = ['hatch', 'sedan', 'suv']

def resize_image(img, size=(64,64)):

    h, w = img.shape[:2]

    if h == w: 
        return cv2.resize(img, size, cv2.INTER_AREA)

    dif = h if h > w else w

    interpolation = cv2.INTER_AREA if dif > (size[0]+size[1])//2 else cv2.INTER_CUBIC

    x_pos = (dif - w)//2
    y_pos = (dif - h)//2

    if len(img.shape) == 2:
        mask = np.zeros((dif, dif), dtype=img.dtype)
        mask[y_pos:y_pos+h, x_pos:x_pos+w] = img[:h, :w]
    else:
        c = img.shape[2]
        mask = np.zeros((dif, dif, c), dtype=img.dtype)
        mask[y_pos:y_pos+h, x_pos:x_pos+w, :] = img[:h, :w, :]

    return cv2.resize(mask, size, interpolation)

spath, dpath = join('data', 'cars'), join('data', 'norm')

if not os.path.isdir(dpath):
    os.mkdir(dpath)

imgs = {}


for ct in car_types:
    sp, dp = join(spath, ct), join(dpath, ct)
    alen = 1024
    imgs[ct] = np.zeros((1024, 64, 64, 3))
    if not os.path.isdir(dp):
        os.mkdir(dp)
    tlen = 0
    for f in listdir(sp):
        sf, df = join(sp, f), join(dp, f)
        img = cv2.imread(sf)
        img = resize_image(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        tlen += 1
        if tlen > alen:
            imgs[ct] = np.append(imgs[ct], np.zeros((1024, 64, 64, 3)))
            alen += 1024
        imgs[ct][tlen-1] = img
    imgs[ct] = imgs[ct][:tlen]
    print(imgs[ct].shape)

In [ ]:
#%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

from tensorflow.keras import utils 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

trainx, testx = imgs['hatch'][:300,:,:,:], imgs['hatch'][300:,:,:,:]

trainy, testy = np.zeros(trainx.shape[0], dtype=float), np.zeros(testx.shape[0], dtype=float)

trainx = np.append(trainx, imgs['sedan'][:375,:,:,:], axis=0)
testx = np.append(testx, imgs['sedan'][375:,:,:,:], axis=0)

ltrain, ltest = trainx.shape[0] - trainy.shape[0], testx.shape[0] - testy.shape[0]

trainy, testy = np.append(trainy, np.ones(ltrain, dtype=float)), np.append(testy, np.ones(ltest, dtype=float))


print(trainx.shape, "y", trainy.shape)
print(testx.shape, 'y', testy.shape)

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(trainx)

trainX, trainY = trainx, trainy
testX, testY   = testx, testy

for i in range(9):
    iterate = datagen.flow(trainx, trainy, batch_size=len(trainx), shuffle=True)
    x, y = iterate.next()
    trainX, trainY = np.append(trainX, x, axis=0), np.append(trainY, y, axis=0)

for i in range(9):
    iterate = datagen.flow(testx, testy, batch_size=len(testx), shuffle=True)
    x, y = iterate.next()
    testX, testY = np.append(testX, x, axis=0), np.append(testY, y, axis=0)


print(trainX.shape, "y", trainY.shape)
print(testX.shape, "y", testY.shape)

trainx, trainy = trainX, trainY
testx, testy   = testX, testY

trainx = trainx.astype('float32') / 255
testx  = testx.astype('float32') / 255

trainx_mean = np.mean(trainx, axis=(0, 1, 2))
trainx_std  = np.std(trainx, axis=(0, 1, 2))

print(trainx_mean, trainx_std)

trainx -= trainx_mean
trainx /= trainx_std

testx -= trainx_mean
testx /= trainx_std

trainX, trainY = trainx, trainy #utils.to_binary(trainy)
testX,  testY  = testx,  testy  #utils.to_binary(testy)

min_pix, max_pix = trainX.min(), trainX.max()

print(min_pix, max_pix)
print(testX.min(), testX.max())

print(trainX.shape, trainY.shape)

In [4]:
print(testY)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.
 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0.
 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1.
 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0.
 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1.
 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1.
 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1.
 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1.
 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0.
 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1.
 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0.

In [5]:
from tensorflow.keras.layers import BatchNormalization, Conv2D, Activation, MaxPool2D
from tensorflow.keras.layers import add, Input, Dense, Flatten, GlobalAvgPool2D, GlobalAvgPool1D
from tensorflow.keras.initializers import zeros

def ResConv(x, kernel=(3, 3), depth=32, maxpool=False):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    if maxpool :
        x = MaxPool2D()(x)
    x = Conv2D(depth, kernel, padding='same', use_bias=False)(x)
    return x

def ResUnit(x, depth=32, maxpool=False):
    x = ResConv(x, depth=depth, maxpool=maxpool)
    x = ResConv(x, depth=depth)
    return x
    
def ResNetBlock(x, nunit, depth=32, maxpool=False):
    assert nunit > 0, "Ensure there are at least 1 unit in the ResNet Block"
    nunit -= 1
    if maxpool:
        xskip = Conv2D(depth, (1, 1), strides=2, use_bias=False)(x)
    else: 
        xskip = x
    x = add([ResUnit(x, depth=depth, maxpool=maxpool), xskip])
    if nunit >= 1:
        nunit -= 1
        for i in range(nunit):
            x = add([ResUnit(x, depth=depth), x])
        x = add([ResUnit(x, depth=depth), x])
    return x

# Returns latent vector of 32 bytes
def ResNet9(x):
    x = Conv2D(64, (7, 7), strides=2, padding='same', use_bias=False)(x)
    x = MaxPool2D((3, 3),  strides=2, padding='same')(x)
    
    nunits   = (2, 3, 2)
    maxpools = (False, True, True)
    depths   = (64, 32, 32)
    
    for i in range(3):
        x = ResNetBlock(x, nunits[i], depth=depths[i], maxpool=maxpools[i])
    x = GlobalAvgPool2D()(x)
    return x

#def D_init():
#    xin = Input(shape=(64, 64, 3))
#    x = ResNet9(xin)
#    x = Dense(1, use_bias=False, activation='sigmoid')(x)
#    return Model(xin, x)

def Q_init():
    xin = Input(shape=(64, 64, 3))
    x = ResNet9(xin)
    return Model(xin, x)


def D_init():
    xin = Input(shape=(64, 64, 3))
    x = Flatten()(xin)
    x = Dense(1, activation='sigmoid')(x)
    return Model(xin, x)
    

In [6]:
from tensorflow.keras import Model
from tensorflow.keras.layers import UpSampling2D, Reshape, Conv2DTranspose

def InvResConv(x, kernel=(3, 3), depth=32, upscale=False):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    if upscale :
        x = UpSampling2D((2,2))(x)
    x = Conv2D(depth, kernel, padding='same', use_bias=False)(x)
    return x

def InvResUnit(x, depth=32, upscale=False):
    x = InvResConv(x, depth=depth)
    x = InvResConv(x, depth=depth, upscale=upscale)
    return x
    
def InvResNetBlock(x, nunit, depth=32, upscale=False):
    assert nunit > 0, "Ensure there are at least 1 unit in the ResNet Block"
    nunit -= 1
    x = InvResUnit(x, depth=depth, upscale=upscale)
    if nunit >= 1:
        nunit -= 1
        for i in range(nunit):
            x = InvResUnit(x, depth=depth)
        x = InvResUnit(x, depth=depth)
    return x


def InvResNet9(x, size=(4,4)):
    depths   = (32, 32, 64)
    upscales = (True, True, False)
    nunits   = (2, 3, 2)

    x = UpSampling2D(size)(x)
    for i in range(3):
        x = InvResNetBlock(x, nunits[i], depth=depths[i], upscale=upscales[i])
    x = UpSampling2D((2,2))(x)
    x = Conv2DTranspose(3, (7, 7), strides=2, padding='same', use_bias=False)(x)
    return x

In [7]:
def G_init():
    xin = Input(shape=(32,), name="Input")
    x = Reshape((1,1,32))(xin)
    x = InvResNet9(x)
    x = Activation('relu')(x)
    model = Model(xin, x)
    return model

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = 8 #int(np.sqrt(s / r))
            h = 8 #int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = 0.0 #np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

datagen = ImageDataGenerator(
        fill_mode = 'constant',
        cval=0,
        #rotation_range=20,
        #width_shift_range=8,  # randomly shift images horizontally (fraction of total width)
        #height_shift_range=8,  # randomly shift images vertically (fraction of total height)
        #horizontal_flip=True,  # randomly flip images
        preprocessing_function=get_random_eraser(v_l=min_pix, v_h=max_pix, pixel_level=False)
)
datagen.fit(trainX)

In [14]:
import matplotlib.pyplot as plt
%matplotlib inline

def accuracy(test_x, test_y, model):
    result = model.predict(test_x)
    predicted_class = np.argmax(result)
    true_class = np.argmax(test_y)
    num_correct = np.sum(predicted_class == true_class) 
    accuracy = float(num_correct)/result.shape[0]
    return (accuracy * 100)

def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['bacc'])+1),model_history.history['bacc'])
    axs[0].plot(range(1,len(model_history.history['val_bacc'])+1),model_history.history['val_bacc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['bacc'])+1),len(model_history.history['bacc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [15]:
BATCH_SIZE = 64
WT_DECAY   = 1e-5
MOMENTUM   = 0.90
LEARNING_RATE = 0.002

from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import categorical_crossentropy, binary_crossentropy
from tensorflow.keras import backend as K

def l2_weights(model):
    l2 = 0
    for layer in model.layers: 
        wt = layer.weights
        if len(wt) > 0:
            l2 += K.sum(K.pow(wt, 2))
    return l2

def reg_loss(model):
    def rloss(y_true, y_pred):
        return WT_DECAY*l2_weights(model)
    return rloss

def loss_with_regularization(model):
    def loss(y_true, y_pred):
        return binary_crossentropy(y_true, y_pred, True) + reg_loss(model)(y_true, y_pred)
    return loss

def bacc(y_true, y_pred):
    y_pred = K.cast(y_pred > 0.5, dtype=y_true.dtype)
    return K.mean(K.equal(y_true, y_pred), axis=-1)

Q = Q_init()
optimizer = SGD(lr=LEARNING_RATE, momentum=MOMENTUM, nesterov=True)
Q.compile(optimizer=optimizer, loss=loss_with_regularization(Q), metrics=[bacc, reg_loss(Q)])
Q.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_31 (Conv2D)              (None, 32, 32, 64)   9408        input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 16, 16, 64)   0           conv2d_31[0][0]                  
__________________________________________________________________________________________________
batch_normalization_28 (BatchNo (None, 16, 16, 64)   256         max_pooling2d_3[0][0]            
____________________________________________________________________________________________

In [16]:
D = D_init()

D.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['acc'])
D.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12288)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 12289     
Total params: 12,289
Trainable params: 12,289
Non-trainable params: 0
_________________________________________________________________


In [17]:
G = G_init()

G.compile(optimizer=Adam(0.0002, 0.5), loss='mse', metrics=['acc'])
G.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 32)]              0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 1, 1, 32)          0         
_________________________________________________________________
up_sampling2d_4 (UpSampling2 (None, 4, 4, 32)          0         
_________________________________________________________________
batch_normalization_42 (Batc (None, 4, 4, 32)          128       
_________________________________________________________________
activation_43 (Activation)   (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 4, 4, 32)          9216      
_________________________________________________________________
batch_normalization_43 (Batc (None, 4, 4, 32)          128 

In [19]:
for i in range(2000):
    Q.fit(trainX, trainY, batch_size=BATCH_SIZE, validation_data = (testX, testY))
    Gin = K.function(Q.input, Q.layers[-1].output)(trainX)
    G.fit(Gin, trainX, batch_size=BATCH_SIZE)
    Gout = G.predict(Gin)
    Dx = np.append(trainX, Gout, axis=0)
    Dy = np.append(np.ones((trainX.shape[0],)), np.zeros((Gout.shape[0],)))
    s = np.arange(Dy.shape[0])
    np.random.shuffle(s)
    Dx, Dy = Dx[s], Dy[s]
    D.fit(Dx, Dy, batch_size=BATCH_SIZE, validation_data = (testX, np.ones((testX.shape[0],))))

Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 3s 371us/sample - loss: 0.9378 - acc: 0.5232
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 96us/sample - loss: 0.4059 - acc: 0.9720 - val_loss: 0.1761 - val_acc: 0.9269
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 268us/sample - loss: 0.8739 - acc: 0.5593
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 53us/sample - loss: 0.3929 - acc: 0.9375 - val_loss: 0.3674 - val_acc: 0.9045
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 267us/sample - loss: 0.8960 - acc: 0.5632
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 50us/sample - loss: 0.3009 - acc: 0.9542 - val_loss: 0.3988 - val_acc: 0.9000
Train on 6750 samples, validate on 670 samples
6750/6750 [===

6750/6750 [==============================] - 2s 268us/sample - loss: 0.8941 - acc: 0.5553
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 53us/sample - loss: 0.1769 - acc: 0.9685 - val_loss: 0.3766 - val_acc: 0.9179
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 270us/sample - loss: 0.9013 - acc: 0.5541
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 50us/sample - loss: 0.1761 - acc: 0.9733 - val_loss: 0.4279 - val_acc: 0.9090
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 271us/sample - loss: 0.9091 - acc: 0.5489
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 54us/sample - loss: 0.1638 - acc: 0.9719 - val_loss: 0.5334 - val_acc: 0.9045
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 268us/sample 

6750/6750 [==============================] - 2s 269us/sample - loss: 0.8849 - acc: 0.5625
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 53us/sample - loss: 0.1625 - acc: 0.9711 - val_loss: 0.5186 - val_acc: 0.8881
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 268us/sample - loss: 0.8877 - acc: 0.5601
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 55us/sample - loss: 0.1419 - acc: 0.9761 - val_loss: 0.4831 - val_acc: 0.9015
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 267us/sample - loss: 0.8977 - acc: 0.5567
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 54us/sample - loss: 0.1421 - acc: 0.9789 - val_loss: 0.3913 - val_acc: 0.9224
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 268us/sample 

6750/6750 [==============================] - 2s 266us/sample - loss: 0.8832 - acc: 0.5615
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 53us/sample - loss: 0.1435 - acc: 0.9758 - val_loss: 0.4110 - val_acc: 0.9090
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 266us/sample - loss: 0.8791 - acc: 0.5648
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 52us/sample - loss: 0.1217 - acc: 0.9793 - val_loss: 0.5382 - val_acc: 0.9045
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 265us/sample - loss: 0.8820 - acc: 0.5635
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 50us/sample - loss: 0.1246 - acc: 0.9750 - val_loss: 0.4164 - val_acc: 0.8955
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 265us/sample 

6750/6750 [==============================] - 2s 270us/sample - loss: 0.8709 - acc: 0.5696
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 54us/sample - loss: 0.1298 - acc: 0.9779 - val_loss: 0.3612 - val_acc: 0.9045
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 267us/sample - loss: 0.8688 - acc: 0.5700
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 51us/sample - loss: 0.1102 - acc: 0.9774 - val_loss: 0.4835 - val_acc: 0.9104
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 269us/sample - loss: 0.8683 - acc: 0.5681
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 53us/sample - loss: 0.1166 - acc: 0.9782 - val_loss: 0.5000 - val_acc: 0.9000
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 270us/sample 

...

6750/6750 [==============================] - 2s 264us/sample - loss: 0.7649 - acc: 0.6000
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 51us/sample - loss: 0.0041 - acc: 0.9993 - val_loss: 1.4576 - val_acc: 0.8552
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 264us/sample - loss: 0.7642 - acc: 0.6035
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 51us/sample - loss: 0.0037 - acc: 0.9997 - val_loss: 1.4293 - val_acc: 0.8701
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 265us/sample - loss: 0.7672 - acc: 0.6013
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 51us/sample - loss: 0.0035 - acc: 0.9997 - val_loss: 1.3947 - val_acc: 0.8716
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 265us/sample 

6750/6750 [==============================] - 2s 264us/sample - loss: 0.7674 - acc: 0.6010
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 51us/sample - loss: 0.0038 - acc: 0.9996 - val_loss: 1.3101 - val_acc: 0.8687
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 266us/sample - loss: 0.7661 - acc: 0.6016
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 51us/sample - loss: 0.0052 - acc: 0.9995 - val_loss: 1.3842 - val_acc: 0.8701
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 266us/sample - loss: 0.7639 - acc: 0.6030
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 50us/sample - loss: 0.0027 - acc: 1.0000 - val_loss: 1.4219 - val_acc: 0.8672
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 263us/sample 

6750/6750 [==============================] - 2s 265us/sample - loss: 0.7674 - acc: 0.6016
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 52us/sample - loss: 0.0040 - acc: 0.9996 - val_loss: 1.3716 - val_acc: 0.8642
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 268us/sample - loss: 0.7663 - acc: 0.6033
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 48us/sample - loss: 0.0060 - acc: 0.9989 - val_loss: 1.4120 - val_acc: 0.8716
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 267us/sample - loss: 0.7638 - acc: 0.6011
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 50us/sample - loss: 0.0073 - acc: 0.9985 - val_loss: 1.4065 - val_acc: 0.8642
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 265us/sample 

6750/6750 [==============================] - 2s 267us/sample - loss: 0.7663 - acc: 0.6014
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 49us/sample - loss: 0.0063 - acc: 0.9987 - val_loss: 1.4277 - val_acc: 0.8672
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 266us/sample - loss: 0.7657 - acc: 0.6016
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 52us/sample - loss: 0.0035 - acc: 0.9997 - val_loss: 1.3915 - val_acc: 0.8731
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 267us/sample - loss: 0.7654 - acc: 0.6036
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 52us/sample - loss: 0.0047 - acc: 0.9995 - val_loss: 1.4606 - val_acc: 0.8627
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 268us/sample 

6750/6750 [==============================] - 2s 267us/sample - loss: 0.7638 - acc: 0.6010
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 50us/sample - loss: 0.0037 - acc: 0.9996 - val_loss: 1.4769 - val_acc: 0.8627
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 269us/sample - loss: 0.7638 - acc: 0.6032
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 51us/sample - loss: 0.0039 - acc: 0.9995 - val_loss: 1.3815 - val_acc: 0.8672
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 267us/sample - loss: 0.7656 - acc: 0.6025
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 51us/sample - loss: 0.0029 - acc: 0.9999 - val_loss: 1.3919 - val_acc: 0.8761
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 264us/sample 